In [1]:
import torch
from hydra import initialize, compose
from hydra.utils import instantiate
from src.data import DataHandler
from src.model import ResNet, ConvNext, Dummy, ModelTrainer
from src.data.dataset import CellPaintingDatasetCached

In [3]:
import os
os.chdir('/home/azureuser/cloudfiles/code/Users/maciej.filanowicz/CellPainting-2.0/src')
initialize(version_base="1.2", config_path="src/conf", job_name="test_app")
cfg = compose(config_name="config", return_hydra_config=True)
dataset_config = instantiate(cfg.dataset)
train_config = instantiate(cfg.train)

In [7]:
# data_loaders = DataHandler(dataset_config).get_data_loaders()
import pandas as pd
metadata = pd.read_csv('splitted_metadata.csv')
dataset_config.dataset_path = dataset_config.dataset_path.replace('raw','processed')
train_dataset = CellPaintingDatasetCached(metadata[metadata.subset=='TRAIN'], dataset_config, dataset_config.transforms)
test_dataset = CellPaintingDatasetCached(metadata[metadata.subset=='TEST'], dataset_config, dataset_config.transforms)
val_dataset = CellPaintingDatasetCached(metadata[metadata.subset=='VAL'], dataset_config, dataset_config.transforms)
train_data_loader = torch.utils.data.DataLoader(train_dataset,dataset_config.batch_size,num_workers=8,pin_memory=True)
test_data_loader = torch.utils.data.DataLoader(test_dataset,dataset_config.batch_size,num_workers=8,pin_memory=True)
val_data_loader = torch.utils.data.DataLoader(val_dataset,dataset_config.batch_size,num_workers=8,pin_memory=True)
data_loaders = {'train_data_loader':train_data_loader,'test_data_loader':test_data_loader,'val_data_loader':val_data_loader}

In [8]:
model = torch.hub.load("pytorch/vision", "convnext_tiny")

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.device_count() > 1:
  print("Let's use", torch.cuda.device_count(), "GPUs!")

Let's use 4 GPUs!


In [9]:
model = Dummy()
trainer = ModelTrainer(model,train_config,**data_loaders)

In [10]:
trainer.run_trainer()

Progress:   0%|          | 0/1 [00:00<?, ?it/s]

Training:   0%|          | 0/162 [00:00<?, ?it/s]